# Лабораторна робота №4
## Виконав: Пєшков Антон, студент КМ-12 ФПМ
1. Відкрити та зчитати наданий файл з даними.
2. Визначити та вивести кількість записів.
3. Вивести атрибути набору даних.
4. З’ясувати збалансованість набору даних.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

from math import sqrt
from sklearn.model_selection import ShuffleSplit
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (recall_score, matthews_corrcoef, precision_score,
                             accuracy_score, f1_score, balanced_accuracy_score)


plt.rcParams['font.family'] = 'Century Gothic'
plt.rcParams['figure.dpi'] = 100
plt.rcParams['axes.titlesize'] = 20
plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['figure.facecolor'] = '#e3f3fc'
COLORS = ['#00a876', '#edb724']


def _wrap(f): return lambda yt, yp: f(yt, yp, average="weighted")


METRICS = {
    accuracy_score: 'Accuracy',
    _wrap(
        lambda *x, **y: precision_score(*x, **y, zero_division=0)
    ): 'Precision',
    _wrap(recall_score): 'Recall',
    matthews_corrcoef: 'MCC',
    _wrap(f1_score): 'F1 score',
    balanced_accuracy_score: 'Bal. accuracy'
}

In [ ]:
df = pd.read_csv("dataset2_l4.txt")

print("Кількість записів:", len(df))
print("Атрибути:", *df.columns)
print("Кількості класів:")
print(df["Class"].value_counts().to_string(header=0))

5. Отримати двадцять варіантів перемішування набору даних та розділення його на навчальну ( тренувальну) та тестову вибірки, використовуючи функцію ShuffleSplit. Сформувати начальну та тестові вибірки на основі обраного користувачем варіанту.

In [ ]:
split = ShuffleSplit(20, test_size=0.2, random_state=1337)

X, Y = df.iloc[:, df.columns != "Class"], df.Class

train_inds, test_inds = next(split.split(X, Y))

X_TRAIN, Y_TRAIN = X.iloc[train_inds], Y.iloc[train_inds]
X_TEST, Y_TEST = X.iloc[test_inds], Y.iloc[test_inds]

6. Використовуючи функцію KNeighborsClassifier бібліотеки scikit-learn, збудувати класифікаційну модель на основі методу k найближчих сусідів (кількість сусідів обрати самостійно, вибір аргументувати) та навчити її на тренувальній вибірці, вважаючи, що цільова характеристика визначається стовпчиком Class, а всі інші виступають в ролі вихідних аргументів.

In [ ]:
# def find_i_optimal():
#     KS = [*range(2, 20)]
#     MX = {name: (k, []) for k, name in METRICS.items()}

#     for k in KS:
#         y_pred = KNeighborsClassifier(k)\
#             .fit(X_TRAIN, Y_TRAIN)\
#             .predict(X_TEST)

#         [vals.append(metric(Y_TEST, y_pred)) for (metric, vals) in MX.values()]

#     [plt.plot(KS, y, label=label) for label, (_, y) in MX.items()]
#     plt.legend()
#     return pd.DataFrame(
#         {name: a for (name, (_, a)) in MX.items()}
#         | {"k": KS}
#     ).set_index("k")

# x = find_i_optimal()
# display(x.head(5))
# k = x.idxmax().mode()[0]
k = 3

model = KNeighborsClassifier(k)
model.fit(X_TRAIN, Y_TRAIN)

In [ ]:
def plot_metrics(x, y_true, title):
    y_pred = model.predict(x)
    metrics = {
        'id': [*METRICS.values()],
        'value': [metric(y_true, y_pred) for metric in METRICS]
    }
    xs = range(len(METRICS))
    ax = plt.subplot()
    ax.bar(xs, metrics['value'], color=COLORS[1])
    [
        ax.text(xi, hi, f"{hi:.3f}", ha="center", va="bottom")
        for xi, hi in zip(xs, metrics['value'])
    ]
    ax.set_title(title)
    ax.set_ylim(0, 1.02)
    ax.set_xticks(xs, metrics['id'])

7. Обчислити класифікаційні метрики збудованої моделі для тренувальної та тестової вибірки. Представити результати роботи моделі на тестовій вибірці графічно.

In [ ]:
plot_metrics(X_TRAIN, Y_TRAIN,
             'Метрики ефективності класифікатора на тренувальній вибірці')

In [ ]:
plot_metrics(X_TEST, Y_TEST,
             'Метрики ефективності класифікатора на тестовій вибірці')

8. Обрати алгоритм KDTree та з’ясувати вплив розміру листа (від 20 до  200 з кроком 5) на результати класифікації. Результати представити графічно.

In [ ]:
def leaf_size_impact():
    mx = {p: [] for p in METRICS.items()}

    for ls in range(20, 201, 5):
        mod = KNeighborsClassifier(3, algorithm='kd_tree', leaf_size=ls)
        mod.fit(X_TEST, Y_TEST)

        y_pred = mod.predict(X_TEST)
        [arr.append(meth(Y_TEST, y_pred)) for (meth, _), arr in mx.items()]

    xs = range(20, 201, 5)
    ax = plt.subplot()
    ax.set_title("Метрики класифікатора в залежності від розміру листа")
    [ax.plot(xs, y, label=label) for (_, label), y in mx.items()]
    ax.legend()

leaf_size_impact()